# 載入函式庫

In [ ]:

from scipy.stats import skew
import numpy as np
import pandas as pd
import librosa


from sklearn.multiclass import  OneVsOneClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from keras.utils import np_utils

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.multiclass import OneVsOneClassifier


import numpy as np
import pandas as pd

# 資料探勘

In [ ]:
from google.colab import drive
drive.mount("/content/nnndrive")

Mounted at /content/nnndrive


In [ ]:
%cd /content/nnndrive/MyDrive/

/content/nnndrive/MyDrive


In [ ]:
%cd /content/nnndrive/MyDrive/voice-demo

/content/nnndrive/MyDrive/voice-demo


In [ ]:
df_csv = pd.read_csv("Training Dataset/training datalist.csv")
print("資料資訊")
df_csv.info()

資料資訊
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID                         1000 non-null   object 
 1   Sex                        1000 non-null   int64  
 2   Age                        1000 non-null   int64  
 3   Disease category           1000 non-null   int64  
 4   Narrow pitch range         1000 non-null   int64  
 5   Decreased volume           1000 non-null   int64  
 6   Fatigue                    1000 non-null   int64  
 7   Dryness                    1000 non-null   int64  
 8   Lumping                    1000 non-null   int64  
 9   heartburn                  1000 non-null   int64  
 10  Choking                    1000 non-null   int64  
 11  Eye dryness                1000 non-null   int64  
 12  PND                        1000 non-null   int64  
 13  Smoking                    1000 non-null   i

# 醫學資料

**資料處理**

In [ ]:
#sex轉成0 or 1

def replace_sex(x):
    return x - 1


df_csv["Sex"] = df_csv["Sex"].apply(replace_sex)

#df(nan ==> 0)

df_csv["PPD"].fillna(0, inplace=True)
df_csv["Voice handicap index - 10"].fillna(0, inplace=True)


#normalization

def replace_nor_age(x):
    return x / 50

def replace_nor_index(x):
    return x / 40

df_csv["Voice handicap index - 10"] = df_csv["Voice handicap index - 10"].apply(replace_nor_index)

df_csv["Age"] = df_csv["Age"].apply(replace_nor_age)


**切分資料**

In [ ]:
y = df_csv["Disease category"].to_numpy().reshape(-1, 1)
print("資料label總數(矩陣): ", y.shape)
df_csv_1 = df_csv.drop(["Disease category","ID"],axis=1)

x = df_csv_1.to_numpy()


y_one_hot = np_utils.to_categorical(y)

y_one_hot = np.argmax(y_one_hot, axis=1)

print(y_one_hot)

x_train, x_test, y_train, y_test = train_test_split(x, y_one_hot, test_size=0.8, random_state=42)

資料label總數(矩陣):  (1000, 1)
[1 2 2 1 1 1 3 2 1 1 1 1 5 1 4 3 3 1 5 1 3 1 1 3 1 2 2 1 5 1 5 2 1 1 3 4 2
 2 1 5 2 2 2 1 5 1 3 1 3 1 1 2 1 3 4 1 1 1 1 1 3 2 4 1 1 1 1 4 3 1 1 1 4 3
 3 2 1 1 1 3 1 1 3 4 1 1 5 3 3 1 2 1 3 3 3 2 1 2 2 1 2 2 1 1 3 2 1 2 3 1 3
 1 2 1 1 1 1 2 1 1 1 1 1 2 3 1 1 1 5 1 5 1 1 1 1 1 1 4 3 1 1 2 2 5 1 1 1 3
 3 2 3 3 3 3 1 2 2 2 2 2 1 1 1 2 1 1 3 2 1 2 1 3 1 2 2 1 4 3 1 2 1 2 4 2 5
 1 2 1 4 1 1 2 2 1 1 5 1 1 1 1 1 3 1 2 1 4 1 3 2 2 1 1 5 1 1 2 3 2 2 2 1 1
 1 2 1 3 1 5 1 1 2 2 3 1 1 1 3 2 1 1 4 3 2 1 4 1 3 1 2 1 1 1 1 2 3 1 1 3 1
 3 1 1 2 5 2 1 1 1 1 2 1 1 2 1 3 1 2 1 1 2 2 3 2 1 3 2 2 1 1 3 2 3 2 1 1 1
 5 3 2 1 1 4 2 5 1 3 1 3 1 1 1 1 2 1 2 1 1 2 1 5 1 1 3 1 3 1 2 1 2 5 3 1 2
 1 1 4 1 3 1 1 1 1 2 1 1 2 1 2 2 1 1 1 1 3 1 4 1 1 1 1 1 1 2 1 3 2 1 1 1 1
 1 1 1 1 2 3 3 1 1 3 5 2 1 2 3 1 1 3 1 4 1 1 1 2 1 2 1 2 4 3 2 1 1 2 1 1 3
 2 1 2 1 1 2 2 2 1 2 1 2 2 1 1 1 1 1 3 1 2 1 4 1 1 1 3 1 1 3 1 2 1 1 3 1 1
 2 2 1 2 1 1 3 1 3 3 3 2 2 1 2 1 3 2 1 1 1 2 1 2 3 1 2 1 1 1 2 3 3 1 3 1 4

**帶入svm**

In [ ]:
clf = OneVsOneClassifier(SVC(kernel="linear"))

clf.fit(x_train,y_train)

y_pred = clf.predict(x)


#=============把資料丟入第2階段==============

medical_x = y_pred.reshape(-1,1)

print(medical_x)

[[1]
 [2]
 [1]
 [1]
 [2]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [1]
 [2]
 [1]
 [1]
 [1]
 [2]
 [1]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [1]
 [1]
 [2]
 [3]
 [1]
 [1]
 [2]
 [1]
 [3]
 [1]
 [3]
 [1]
 [2]
 [2]
 [1]
 [3]
 [1]
 [1]
 [2]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [3]
 [1]
 [2]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [4]
 [1]
 [1]
 [2]
 [1]
 [3]
 [1]
 [2]
 [1]
 [1]
 [2]
 [2]
 [1]
 [2]
 [2]
 [1]
 [1]
 [2]
 [1]
 [1]
 [2]
 [3]
 [1]
 [1]
 [1]
 [2]
 [1]
 [1]
 [1]
 [1]
 [2]
 [1]
 [1]
 [2]
 [1]
 [1]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [4]
 [1]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [3]
 [3]
 [1]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [1]
 [1]
 [1]
 [2]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [1]
 [2]
 [1]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]


# 音訊資料

**引入音訊**

In [ ]:

# 挑選出要訓練的Disease category 1、2、3
df_csv = df_csv.loc[df_csv['Disease category'].isin([1, 2, 3, 4, 5]), ['ID', 'Disease category']]

# 在dataframe中加入要訓練的音檔路徑
df_csv['wav_path'] = df_csv['ID'].map("./Training Dataset/training_voice_data/{}.wav".format)

print("Disease category in source_df :",df_csv['Disease category'].unique())
print("source_df :\n", df_csv)

Disease category in source_df : [1 2 3 5 4]
source_df :
           ID  Disease category  \
0    1202f15                 1   
1    0600ve0                 2   
2    1001o7l                 2   
3    1201c1t                 1   
4    0402jvt                 1   
..       ...               ...   
995  0G00ftn                 3   
996  1201pkr                 4   
997  0202p64                 3   
998  12021au                 2   
999  04027it                 2   

                                              wav_path  
0    ./Training Dataset/training_voice_data/1202f15...  
1    ./Training Dataset/training_voice_data/0600ve0...  
2    ./Training Dataset/training_voice_data/1001o7l...  
3    ./Training Dataset/training_voice_data/1201c1t...  
4    ./Training Dataset/training_voice_data/0402jvt...  
..                                                 ...  
995  ./Training Dataset/training_voice_data/0G00ftn...  
996  ./Training Dataset/training_voice_data/1201pkr...  
997  ./Training Datas

**定義mfcc函式**

In [ ]:
# 定義函數
def audio_to_mfccs(filename, sample_rate=44100, offset=0, duration=None):
    # 讀取音訊檔案，並設定取樣率、起始時間、及持續時間
    voice, sample_rate = librosa.load(
        filename, sr=sample_rate, offset=offset, duration=duration
    )

    # 將時間值轉換為 FFT 與 hop length 所需的框架數 (以取樣點表示)
    n_fft = int(16/1000 * sample_rate)  # 將 16 毫秒轉換為取樣點
    hop_length = int(8/1000 * sample_rate)  # 將 8 毫秒轉換為取樣點

    # 計算音訊數據的 MFCC 特徵
    mfcc_feature = librosa.feature.mfcc(
        y=voice, sr=sample_rate, n_mfcc=13, n_fft=n_fft, hop_length=hop_length)

    # 計算 MFCC 的一階和二階差分特徵
    delta_mfcc_feature = librosa.feature.delta(mfcc_feature)

    # 將原始 MFCC 特徵和差分特徵串聯起來，得到所有幀的特徵向量
    mfccs = np.concatenate((mfcc_feature, delta_mfcc_feature))
    mfccs_features = np.transpose(mfccs)  # 將矩陣轉置，使每行代表一個幀

    # 返回特徵向量
    return mfccs_features

In [ ]:
training_id = df_csv['ID'].tolist()
training_data = pd.DataFrame()
for id in training_id:
    
    mfccs_feature = audio_to_mfccs(df_csv[df_csv['ID']==id]['wav_path'].values[0])
    df = pd.DataFrame()
    for i in range(26):
        df_i = pd.DataFrame(np.array(mfccs_feature[0][i]).reshape(1,-1))
        df = pd.concat([df, df_i], axis=1)
        
    # print("id :",id, ", number of frames :", df.shape[0])
    # 訓練資料標記
    label = df_csv[df_csv['ID']==id]['Disease category'].values[0]
    if label==1:
        df['c1'] = 1; df['c2'] = 0; df['c3'] = 0 ; df['c4'] = 0; df['c5'] = 0
    elif label==2:
        df['c1'] = 0; df['c2'] = 1; df['c3'] = 0 ; df['c4'] = 0; df['c5'] = 0
    elif label==3:
        df['c1'] = 0; df['c2'] = 0; df['c3'] = 1 ; df['c4'] = 0; df['c5'] = 0
    elif label==4:
        df['c1'] = 0; df['c2'] = 0; df['c3'] = 0 ; df['c4'] = 1; df['c5'] = 0
    elif label==5:
        df['c1'] = 0; df['c2'] = 0; df['c3'] = 0 ; df['c4'] = 0; df['c5'] = 1
    else:
        df['c1'] = np.nan; df['c2'] = np.nan; df['c3'] = np.nan; df['c4'] = np.nan; df['c5'] = np.nan

    training_data = pd.concat([training_data, df])
    
print("training_data.shape :", training_data.shape)

x_autoi = training_data.iloc[:, :-5]
y_autoi = training_data.iloc[:, -5:]
print("x_train.shape, y_train.shape :", x_autoi.shape, y_autoi.shape)
print("y_train.columns :", y_autoi.columns.tolist())

training_data.shape : (1000, 31)
x_train.shape, y_train.shape : (1000, 26) (1000, 5)
y_train.columns : ['c1', 'c2', 'c3', 'c4', 'c5']


In [ ]:
x = x_autoi.values
y = y_autoi.values

y = np.argmax(y, axis=1)

for i in range(len(y)):
    y[i] += 1

In [ ]:
print(y)

[1 2 2 1 1 1 3 2 1 1 1 1 5 1 4 3 3 1 5 1 3 1 1 3 1 2 2 1 5 1 5 2 1 1 3 4 2
 2 1 5 2 2 2 1 5 1 3 1 3 1 1 2 1 3 4 1 1 1 1 1 3 2 4 1 1 1 1 4 3 1 1 1 4 3
 3 2 1 1 1 3 1 1 3 4 1 1 5 3 3 1 2 1 3 3 3 2 1 2 2 1 2 2 1 1 3 2 1 2 3 1 3
 1 2 1 1 1 1 2 1 1 1 1 1 2 3 1 1 1 5 1 5 1 1 1 1 1 1 4 3 1 1 2 2 5 1 1 1 3
 3 2 3 3 3 3 1 2 2 2 2 2 1 1 1 2 1 1 3 2 1 2 1 3 1 2 2 1 4 3 1 2 1 2 4 2 5
 1 2 1 4 1 1 2 2 1 1 5 1 1 1 1 1 3 1 2 1 4 1 3 2 2 1 1 5 1 1 2 3 2 2 2 1 1
 1 2 1 3 1 5 1 1 2 2 3 1 1 1 3 2 1 1 4 3 2 1 4 1 3 1 2 1 1 1 1 2 3 1 1 3 1
 3 1 1 2 5 2 1 1 1 1 2 1 1 2 1 3 1 2 1 1 2 2 3 2 1 3 2 2 1 1 3 2 3 2 1 1 1
 5 3 2 1 1 4 2 5 1 3 1 3 1 1 1 1 2 1 2 1 1 2 1 5 1 1 3 1 3 1 2 1 2 5 3 1 2
 1 1 4 1 3 1 1 1 1 2 1 1 2 1 2 2 1 1 1 1 3 1 4 1 1 1 1 1 1 2 1 3 2 1 1 1 1
 1 1 1 1 2 3 3 1 1 3 5 2 1 2 3 1 1 3 1 4 1 1 1 2 1 2 1 2 4 3 2 1 1 2 1 1 3
 2 1 2 1 1 2 2 2 1 2 1 2 2 1 1 1 1 1 3 1 2 1 4 1 1 1 3 1 1 3 1 2 1 1 3 1 1
 2 2 1 2 1 1 3 1 3 3 3 2 2 1 2 1 3 2 1 1 1 2 1 2 3 1 2 1 1 1 2 3 3 1 3 1 4
 2 2 2 2 1 1 1 1 1 3 3 1 

In [ ]:
new_x = np.zeros((1000, 6))
for i in range(1000):
    # 計算最大值
    max_a = np.max(x[i])

    # 計算最小值
    min_a = np.min(x[i])

    # 計算平均數
    mean_a = np.mean(x[i])

    # 計算中位數
    median_a = np.median(x[i])

    # 計算標準差
    std_a = np.std(x[i])

    # 計算偏度
    skew_a = skew(x[i])

    # 轉換矩陣為一個包含六個元素的陣列
    new_x[i] = np.array([max_a, min_a, mean_a, median_a, std_a, skew_a])
print(new_x)

[[ 1.48494080e+02 -5.53074524e+02 -1.63179493e+01 -5.11784554e-01
   1.11396294e+02 -4.17364942e+00]
 [ 1.32414246e+02 -2.22018585e+02 -6.04307556e+00 -9.04636860e-01
   5.09200935e+01 -2.15934679e+00]
 [ 1.86702820e+02 -3.39410583e+02 -1.00040083e+01 -6.12908483e-01
   7.62460938e+01 -2.44126657e+00]
 ...
 [ 1.13782799e+02 -2.79279510e+02 -9.72987747e+00 -1.34194446e+00
   5.88886871e+01 -3.32959967e+00]
 [ 1.23644386e+02 -4.17048859e+02 -1.30127821e+01 -2.56309092e-01
   8.47630157e+01 -3.99959885e+00]
 [ 1.93103134e+02 -3.73614166e+02 -9.09355068e+00 -3.75599228e-02
   8.26008835e+01 -2.73785199e+00]]


In [ ]:
x_train_2, x_test_2 ,y_train_2, y_test_2 = train_test_split(new_x, y, test_size=0.8, random_state=42)

clf = OneVsOneClassifier(SVC(kernel="linear"))

clf.fit(x_train_2,y_train_2)

y_pred_2 = clf.predict(new_x)


In [ ]:
print(y_pred_2.reshape(-1,1))

[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]


In [ ]:
y_pred_2 = y_pred_2 / 5
y_pred = y_pred / 5

In [ ]:
y_pred_2 = y_pred_2.reshape(-1,1)
y_pred = y_pred.reshape(-1,1)
merged_arr = np.concatenate((y_pred_2, y_pred,x,new_x), axis=1)

print(merged_arr.shape)

(1000, 34)


In [ ]:
x_train_1,x_pred,y_train_1,y_pred = train_test_split(merged_arr, y, test_size=0.25, random_state=42)

print(x_train_1.shape)
print(y_train_1.shape)

print(x_pred.shape)
print(y_pred.shape)

(750, 34)
(750, 1)
(250, 34)
(250, 1)


In [ ]:
import tensorflow as tf

# 定義模型架構
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(34,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])



# 編譯模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train_1,y_train_1,epochs=10)



Epoch 1/10
24/24 [==============================] - 6s 4ms/step - loss: -97.3834 - accuracy: 0.5307
Epoch 2/10
24/24 [==============================] - 0s 4ms/step - loss: -268.8583 - accuracy: 0.5307
Epoch 3/10
24/24 [==============================] - 0s 3ms/step - loss: -627.3952 - accuracy: 0.5307
Epoch 4/10
24/24 [==============================] - 0s 3ms/step - loss: -1253.1206 - accuracy: 0.5307
Epoch 5/10
24/24 [==============================] - 0s 4ms/step - loss: -2266.0039 - accuracy: 0.5307
Epoch 6/10
24/24 [==============================] - 0s 3ms/step - loss: -3814.4827 - accuracy: 0.5307
Epoch 7/10
24/24 [==============================] - 0s 3ms/step - loss: -6046.7280 - accuracy: 0.5307
Epoch 8/10
24/24 [==============================] - 0s 3ms/step - loss: -9005.4795 - accuracy: 0.5307
Epoch 9/10
24/24 [==============================] - 0s 3ms/step - loss: -13047.9824 - accuracy: 0.5307
Epoch 10/10
24/24 [==============================] - 0s 4ms/step - loss: -18234.9199 